In [3]:
import math
from neuroml import NeuroMLDocument
from neuroml import Cell
from neuroml import BiophysicalProperties
from neuroml import MembraneProperties
from neuroml import IncludeType, Include, Property
from neuroml import ChannelDensity, ChannelDensityNonUniform, ChannelDensityNernst, ChannelDensityGHK2
from neuroml import SpikeThresh
from neuroml import SpecificCapacitance
from neuroml import InitMembPotential
from neuroml import IntracellularProperties
from neuroml import Resistivity
from neuroml import Morphology, Segment, Point3DWithDiam, SegmentGroup
from neuroml import Network, Population, InhomogeneousParameter, ProximalDetails
from neuroml import PulseGenerator, ExplicitInput
from neuroml import InhomogeneousParameter, InhomogeneousValue, VariableParameter
from pyneuroml import pynml
import numpy as np
from pyneuroml.lems import LEMSSimulation
from CellBuilder import (create_cell, add_segment, add_channel_density, set_init_memb_potential, set_resistivity, set_specific_capacitance, get_seg_group_by_id)

In [5]:
def main():
    """Main function

    Include the NeuroML model into a LEMS simulation file, run it, plot some
    data.
    """
    # Simulation bits

    sim_id = "pyr_multi_comp"
    simulation = LEMSSimulation(sim_id=sim_id, duration=700, dt=0.01, simulation_seed=123)

    # Include the NeuroML model file
    simulation.include_neuroml2_file(create_pyr_network())
    # Assign target for the simulation
    simulation.assign_simulation_target("multiple_pyr_cell_network")

    # Recording information from the simulation
    simulation.create_output_file(id="output0", file_name=sim_id + ".dat")
    simulation.add_column_to_output_file("output0", column_id="pop0[0]_v", quantity="pop0[0]/v")

    # Recording information from the simulation
    # simulation.create_output_file(id="na_m", file_name=sim_id + ".na_m.dat")
    # simulation.add_column_to_output_file("na_m", column_id="pop0[0]_na_m", quantity="pop0[0]/pyr_b_prop/membraneProperties/nat_channels/nat/m/q")

    # simulation.create_output_file(id="na_h", file_name=sim_id + ".na_h.dat")
    # simulation.add_column_to_output_file("na_h", column_id="pop0[0]_na_h", quantity="pop0[0]/pyr_b_prop/membraneProperties/nat_channels/nat/h/q")

    # simulation.create_output_file(id="kfast_n", file_name=sim_id + ".kfast_n.dat")
    # simulation.add_column_to_output_file("kfast_n", column_id="pop0[0]_kfast_n", quantity="pop0[0]/pyr_b_prop/membraneProperties/kfast_channels/kfast/n/q")

    # simulation.create_output_file(id="nap_m", file_name=sim_id + ".nap_m.dat")
    # simulation.add_column_to_output_file("nap_m", column_id="pop0[0]_nap_m", quantity="pop0[0]/pyr_b_prop/membraneProperties/nap_channels/nap/m/q")

    # simulation.create_output_file(id="kslow_a", file_name=sim_id + ".kslow_a.dat")
    # simulation.add_column_to_output_file("kslow_a", column_id="pop0[0]_kslow_a", quantity="pop0[0]/pyr_b_prop/membraneProperties/kslow_channels/kslow/a/q")

    # simulation.create_output_file(id="kslow_b", file_name=sim_id + ".kslow_b.dat")
    # simulation.add_column_to_output_file("kslow_b", column_id="pop0[0]_kslow_b", quantity="pop0[0]/pyr_b_prop/membraneProperties/kslow_channels/kslow/b/q")

    # simulation.create_output_file(id="ikm_m", file_name=sim_id + ".ikm_m.dat")
    # simulation.add_column_to_output_file("ikm_m", column_id="pop0[0]_ikm_m", quantity="pop0[0]/pyr_b_prop/membraneProperties/km_channels/km/m/q")

    # Save LEMS simulation to file
    sim_file = simulation.save_to_file()

    # Run the simulation using the default jNeuroML simulator
    pynml.run_lems_with_jneuroml(sim_file, max_memory="2G", nogui=True, plot=False)
    # Plot the data
    plot_data(sim_id)

In [6]:
def plot_data(sim_id):
    """Plot the sim data.

    Load the data from the file and plot the graph for the membrane potential
    using the pynml generate_plot utility function.

    :sim_id: ID of simulaton

    """
    data_array = np.loadtxt(sim_id + ".dat")
    pynml.generate_plot([data_array[:, 0]], [data_array[:, 1]], "Membrane potential", show_plot_already=False, save_figure_to=sim_id + "-v.png", xaxis="time (s)", yaxis="membrane potential (V)")

    data_array_na_m = np.loadtxt(sim_id + ".na_m.dat")
    pynml.generate_plot([data_array_na_m[:, 0]], [data_array_na_m[:, 1]], "Nat m gate", show_plot_already=False, save_figure_to=sim_id + "-na_m.png", xaxis="time (s)", yaxis="q")

    data_array_na_h = np.loadtxt(sim_id + ".na_h.dat")
    pynml.generate_plot([data_array_na_h[:, 0]], [data_array_na_h[:, 1]], "Nat h gate", show_plot_already=False, save_figure_to=sim_id + "-na_h.png", xaxis="time (s)", yaxis="q")

    data_array_kfast_n = np.loadtxt(sim_id + ".kfast_n.dat")
    pynml.generate_plot([data_array_kfast_n[:, 0]], [data_array_kfast_n[:, 1]], "Kfast n gate", show_plot_already=False, save_figure_to=sim_id + "-kfast_n.png", xaxis="time (s)", yaxis="q")

    data_array_nap_m = np.loadtxt(sim_id + ".nap_m.dat")
    pynml.generate_plot([data_array_nap_m[:, 0]], [data_array_nap_m[:, 1]], "Nap m gate", show_plot_already=False, save_figure_to=sim_id + "-nap_m.png", xaxis="time (s)", yaxis="q")

    data_array_kslow_a = np.loadtxt(sim_id + ".kslow_a.dat")
    pynml.generate_plot([data_array_kslow_a[:, 0]], [data_array_kslow_a[:, 1]], "Kslow a gate", show_plot_already=False, save_figure_to=sim_id + "-kslow_a.png", xaxis="time (s)", yaxis="q")

    data_array_kslow_b = np.loadtxt(sim_id + ".kslow_b.dat")
    pynml.generate_plot([data_array_kslow_b[:, 0]], [data_array_kslow_b[:, 1]], "Kslow b gate", show_plot_already=False, save_figure_to=sim_id + "-kslow_b.png", xaxis="time (s)", yaxis="q")

    data_array_ikm_m = np.loadtxt(sim_id + ".ikm_m.dat")
    pynml.generate_plot([data_array_ikm_m[:, 0]], [data_array_ikm_m[:, 1]], "IKM m gate", show_plot_already=False, save_figure_to=sim_id + "-ikm_m.png", xaxis="time (s)", yaxis="q")

In [4]:
def create_pyr_cell():
    pyr_cell_doc = NeuroMLDocument(id='cell', notes="Layer 5 Pyramidal cell")
    cell = create_cell("pyr")
    nml_cell_file = cell.id + ".cell.nml"

    pyr_cell_doc.includes.append(IncludeType("kca.channel.nml"))
    pyr_cell_doc.includes.append(IncludeType("sca.channel.nml"))
    # pyr_cell_doc.includes.append(IncludeType("cad.nml"))

    # pyr_cell_doc.includes.append(IncludeType("kslow.channel.nml"))
    # pyr_cell_doc.includes.append(IncludeType("nat.channel.nml"))
    # pyr_cell_doc.includes.append(IncludeType("nap.channel.nml"))
    # pyr_cell_doc.includes.append(IncludeType("IKM.channel.nml"))

    # Add two soma segments
    diam = 23.1453
    soma_0 = add_segment(cell,
                         prox=[0.0, 0.0, 0.0, diam],
                         dist=[11.5726, 0.0, 0.0, diam],
                         name="Seg0_soma",
                         group="soma")

    soma_1 = add_segment(cell,
                         prox=None,
                         dist=[diam, 0.0, 0.0, diam],
                         name="Seg1_soma",
                         parent=soma_0,
                         group="soma")

    # Add hillock segments
    hillock_0 = add_segment(cell,
                            parent=soma_0,
                            prox=[0, 0, 0, 3.35],
                            dist=[-2, 1.74846e-07, 0, 3.35],
                            name="Seg0_hillock",
                            group="hillock")

    hillock_1 = add_segment(cell,
                            parent=hillock_0,
                            prox=None,
                            dist=[-6, 5.24537e-07, 0, 3.05],
                            name="Seg1_hillock",
                            group="hillock")

    hillock_2 = add_segment(cell,
                            parent=hillock_1,
                            prox=None,
                            dist=[-10, 8.74228e-07, 0, 2.75],
                            name="Seg2_hillock",
                            group="hillock")

    hillock_3 = add_segment(cell,
                            parent=hillock_2,
                            prox=None,
                            dist=[-14, 1.22392e-06, 0, 2.45],
                            name="Seg3_hillock",
                            group="hillock")

    hillock_4 = add_segment(cell,
                            parent=hillock_3,
                            prox=None,
                            dist=[-18, 1.57361e-06, 0, 2.15],
                            name="Seg4_hillock",
                            group="hillock"
                            )

    hillock_5 = add_segment(cell,
                            parent=hillock_4,
                            prox=None,
                            dist=[-20, 1.7486e-06, 0, 2.15],
                            name="Seg5_hillock",
                            group="hillock"
                            )

    # Add basal segments
    basal_0 = add_segment(cell,
                          parent=soma_0,
                          prox=[11.5726, 0, 0, 8.74536],
                          dist=[11.5726, 128.5, 0, 8.74536],
                          name="Seg0_basal",
                          group="basal")

    basal_1 = add_segment(cell,
                          parent=basal_0,
                          prox=None,
                          dist=[11.5726, 257, 0, 8.74536],
                          name="Seg1_basal",
                          group="basal") 

    # Add apical segments
    apical_0 = add_segment(cell,
                           parent=soma_1,
                           prox=[23.1453, 0, 0, 5.92845],
                           dist=[73.1453, 0, 0, 5.92845],
                           name="Seg0_apical",
                           group="apical")

    apical_1 = add_segment(cell,
                           parent=apical_0,
                           prox=None,
                           dist=[173.145, 0, 0, 5.92845],
                           name="Seg1_apical",
                           group="apical")

    apical_2 = add_segment(cell,
                          parent=apical_1,
                          prox=None,
                          dist=[273.145, 0, 0, 5.92845],
                          name="Seg2_apical",
                          group="apical")

    apical_3 = add_segment(cell,
                          parent=apical_2,
                          prox=None,
                          dist=[373.145, 0, 0, 5.92845],
                          name="Seg3_apical",
                          group="apical")

    apical_4 = add_segment(cell,
                        parent=apical_3,
                        prox=None,
                        dist=[473.145, 0, 0, 5.92845],
                        name="Seg4_apical",
                        group="apical")

    apical_5 = add_segment(cell,
                        parent=apical_4,
                        prox=None,
                        dist=[523.145, 0, 0, 5.92845],
                        name="Seg5_apical",
                        group="apical")

    # Add iseg segments
    iseg_0 = add_segment(cell,
                        parent=hillock_5,
                        prox=[-20, 1.74846e-06, 0, 1.95],
                        dist=[-22.5, 2.12595e-06, 0, 1.95],
                        name="Seg0_iseg",
                        group="iseg")

    iseg_1 = add_segment(cell,
                        parent=iseg_0,
                        prox=None,
                        dist=[-27.5, 2.88092e-06, 0, 1.85],
                        name="Seg1_iseg",
                        group="iseg")

    iseg_2 = add_segment(cell,
                        parent=iseg_1,
                        prox=None,
                        dist=[-32.5, 3.6359e-06, 0, 1.75],
                        name="Seg2_iseg",
                        group="iseg")

    iseg_3 = add_segment(cell,
                        parent=iseg_2,
                        prox=None,
                        dist=[-37.5, 4.39088e-06, 0, 1.65],
                        name="Seg3_iseg",
                        group="iseg")

    iseg_4 = add_segment(cell,
                        parent=iseg_3,
                        prox=None,
                        dist=[-42.5, 5.14586e-06, 0, 1.55],
                        name="Seg4_iseg",
                        group="iseg")

    iseg_5 = add_segment(cell,
                        parent=iseg_4,
                        prox=None,
                        dist=[-45, 5.52335e-06, 0, 1.55],
                        name="Seg5_iseg",
                        group="iseg")

    # Add tuft segments
    tuft_0 = add_segment(cell,
                        parent=apical_5,
                        prox=[523.145, 0, 0, 6.01807],
                        dist=[647.895, 0, 0, 6.01807],
                        name="Seg0_tuft",
                        group="tuft"
                        )

    tuft_1 = add_segment(cell,
                        parent=tuft_0,
                        prox=None,
                        dist=[897.395, 0, 0, 6.01807],
                        name="Seg1_tuft",
                        group="tuft"
                        )

    tuft_2 = add_segment(cell,
                        parent=tuft_1,
                        prox=None,
                        dist=[1022.15, 0, 0, 6.01807],
                        name="Seg2_tuft",
                        group="tuft"
                        )

    # Add axon segments
    axon_0 = add_segment(cell,
                        parent=iseg_5,
                        prox=[-45, 5.52335e-06, 0, 1.5],
                        dist=[-295, 4.32723e-05, 0, 1.5],
                        name="Seg0_axon",
                        group="axon"
                        )

    axon_1 = add_segment(cell,
                        parent=axon_0,
                        prox=None,
                        dist=[-545, 8.10213e-05, 0, 1.5],
                        name="Seg1_axon",
                        group="axon")

    for section_name in ["soma", "hillock", "basal", "apical", "iseg", "tuft", "axon"]:
        section_group = get_seg_group_by_id(section_name, cell)
        section_group.neuro_lex_id = 'sao864921383'
        if section_name in ["apical", "hillock", "iseg"]:
            section_group.properties.append(Property(tag="numberInternalDivisions", value="5"))
        elif section_name in ["tuft"]:
            section_group.properties.append(Property(tag="numberInternalDivisions", value="2"))
        else:
            continue

    axosomatic = SegmentGroup(neuro_lex_id=None, id="axosomatic_list")
    apicaltree = SegmentGroup(neuro_lex_id=None, id="apicaltree_list")
    axon = SegmentGroup(neuro_lex_id=None, id="axon_list")

    cell.morphology.segment_groups.append(axosomatic)
    cell.morphology.segment_groups.append(apicaltree)
    cell.morphology.segment_groups.append(axon)

    axosomatic_group = get_seg_group_by_id("axosomatic_list", cell)
    axosomatic_group.includes.append(Include(segment_groups="soma"))
    axosomatic_group.includes.append(Include(segment_groups="hillock"))
    axosomatic_group.includes.append(Include(segment_groups="basal"))
    axosomatic_group.includes.append(Include(segment_groups="iseg"))
    axosomatic_group.includes.append(Include(segment_groups="axon"))
    axosomatic_group.properties.append(Property(tag="color", value="0.8 0 0"))

    apicaltree_group = get_seg_group_by_id("apicaltree_list", cell)
    apicaltree_group.includes.append(Include(segment_groups="apical"))
    apicaltree_group.includes.append(Include(segment_groups="tuft"))

    axon_group = get_seg_group_by_id("axon_list", cell)
    axon_group.includes.append(Include(segment_groups="hillock"))
    axon_group.includes.append(Include(segment_groups="iseg"))
    axon_group.includes.append(Include(segment_groups="axon"))


    for g in ['apicaltree_list', 'apical']:
        seg_group = get_seg_group_by_id(g, cell)
        seg_group.inhomogeneous_parameters.append(
            InhomogeneousParameter(
                id="PathLengthOver_" + g,
                variable="p",
                metric="Path Length from root",
                proximal=ProximalDetails(
                    translation_start="0")))

    # channels
    # leak
    add_channel_density(cell, pyr_cell_doc,
                        cd_id="axosomatic_gpas",
                        cond_density="0.485726 S_per_m2",
                        ion_channel="pas",
                        ion_chan_def_file="pas.channel.nml",
                        erev="-80.3987 mV",
                        ion="non_specific",
                        group="axosomatic_list")

    add_channel_density(cell, pyr_cell_doc,
                    cd_id="apicaltree_gpas",
                    cond_density="0.381196 S_per_m2",
                    ion_channel="pas",
                    ion_chan_def_file="pas.channel.nml",
                    erev="-80.3987 mV",
                    ion="non_specific",
                    group="apicaltree_list")                     

    # nat_soma
    add_channel_density(cell, pyr_cell_doc,
                        cd_id="nat_soma",
                        cond_density="236.616175 S_per_m2",
                        ion_channel="nat",
                        ion_chan_def_file="nat.channel.nml",
                        erev="55mV",
                        ion="na",
                        group="soma")

    # nat_hillock
    add_channel_density(cell, pyr_cell_doc,
                    cd_id="nat_hillock",
                    cond_density="9512.289205 S_per_m2",
                    ion_channel="nat",
                    ion_chan_def_file="nat.channel.nml",
                    erev="55mV",
                    ion="na",
                    group="hillock")
    
    # nat_iseg
    add_channel_density(cell, pyr_cell_doc,
                    cd_id="nat_iseg",
                    cond_density="13326.766938 S_per_m2",
                    ion_channel="nat",
                    ion_chan_def_file="nat.channel.nml",
                    erev="55mV",
                    ion="na",
                    group="iseg")

    # nat_tuft
    add_channel_density(cell, pyr_cell_doc,
                    cd_id="nat_tuft",
                    cond_density="47.82 S_per_m2",
                    ion_channel="nat",
                    ion_chan_def_file="nat.channel.nml",
                    erev="55mV",
                    ion="na",
                    group="tuft")

    # kfast_soma
    add_channel_density(cell, pyr_cell_doc,
                        cd_id="kfast_soma",
                        cond_density="67.197508 S_per_m2",
                        ion_channel="kfast",
                        ion_chan_def_file="kfast.channel.nml",
                        erev="-80mV",
                        ion="k",
                        group="soma")

    # kslow_soma
    add_channel_density(cell, pyr_cell_doc,
                        cd_id="kslow_soma",
                        cond_density="475.82 S_per_m2",
                        ion_channel="kslow",
                        ion_chan_def_file="kslow.channel.nml",
                        erev="-80mV",
                        ion="k",
                        group="soma")   

    # nap_soma
    add_channel_density(cell, pyr_cell_doc,
                        cd_id="nap_soma",
                        cond_density="1.44 S_per_m2",
                        ion_channel="nap",
                        ion_chan_def_file="nap.channel.nml",
                        erev="55mV",
                        ion="na",
                        group="soma")  

    # km_soma
    add_channel_density(cell, pyr_cell_doc,
                        cd_id="km_soma",
                        cond_density="10.459916 S_per_m2",
                        ion_channel="km",
                        ion_chan_def_file="IKM.channel.nml",
                        erev="-80mV",
                        ion="k",
                        group="soma")

    # ih_basal
    add_channel_density(cell, pyr_cell_doc,
                        cd_id="ih_basal",
                        cond_density="11.04 S_per_m2",
                        ion_channel="ih",
                        ion_chan_def_file="ih.channel.nml",
                        erev="-80mV",
                        ion="hcn",
                        group="basal")

    # sca_tuft
    sca_tuft = ChannelDensityGHK2(id="tuft_sca", ion_channel="sca", cond_density="0.45423528 S_per_m2", segment_groups="tuft", ion="ca")
    cell.biophysical_properties.membrane_properties.channel_density_ghk2s.append(sca_tuft)

    # kca_tuft
    kca_tuft = ChannelDensityNernst(id="tuft_kca", ion_channel="kca", cond_density="6.15058501 S_per_m2", segment_groups="tuft", ion="k")
    cell.biophysical_properties.membrane_properties.channel_density_nernsts.append(kca_tuft)
                                
    gbar_kfast_apicaltree = VariableParameter(parameter="condDensity",
                                            segment_groups="apicaltree_list",
                                            inhomogeneous_value=InhomogeneousValue(inhomogeneous_parameters="PathLengthOver_apicaltree_list",
                                            value="67.197508*exp(-p/20.075497)"))
    kfast_apicaltree_list = ChannelDensityNonUniform(id="apicaltree_g_kfast", ion_channel="kfast",
                                                    erev="-80 mV", ion="k")
    kfast_apicaltree_list.variable_parameters.append(gbar_kfast_apicaltree)

    gbar_kslow_apicaltree = VariableParameter(parameter="condDensity",
                                            segment_groups="apicaltree_list",
                                            inhomogeneous_value=InhomogeneousValue(inhomogeneous_parameters="PathLengthOver_apicaltree_list",
                                            value="475.820646*exp(-p/37.711817)"))
    kslow_apicaltree_list = ChannelDensityNonUniform(id="apicaltree_g_kslow", ion_channel="kslow", erev="-80mV", ion="k")
    kslow_apicaltree_list.variable_parameters.append(gbar_kslow_apicaltree)


    cell.biophysical_properties.membrane_properties.channel_density_non_uniforms.append(kfast_apicaltree_list)
    cell.biophysical_properties.membrane_properties.channel_density_non_uniforms.append(kslow_apicaltree_list)

    # Other cell properties
    set_specific_capacitance(cell, "2.23041 uF_per_cm2", group="axosomatic_list")
    set_specific_capacitance(cell, "1.750419 uF_per_cm2", group="apicaltree_list")
    set_init_memb_potential(cell, "-67mV")
    set_resistivity(cell, "0.082 kohm_cm")

    # # Define a cell
    # pyr_cell = Cell(id="pyr_cell", notes="A single compartment Layer 5 Pyramidal cell")

    # # Define its biophysical properties
    # bio_prop = BiophysicalProperties(id="pyr_b_prop")
    # #  notes="Biophysical properties for Layer 5 Pyramidal cell")

    # # Membrane properties are a type of biophysical properties
    # mem_prop = MembraneProperties()
    # # Add membrane properties to the biophysical properties
    # bio_prop.membrane_properties = mem_prop

    # # Append to cell
    # pyr_cell.biophysical_properties = bio_prop


    # pas_channel_density = ChannelDensity(id="pas_channels", cond_density="0.485726 S_per_m2", erev="-80.3987 mV", ion="non_specific", ion_channel="pas")
    # mem_prop.channel_densities.append(pas_channel_density)

    # # Channel density for kfast channel
    # kfast_channel_density = ChannelDensity(id="kfast_channels", cond_density="67.197508 S_per_m2", erev="-80 mV", ion="k", ion_channel="kfast")
    # mem_prop.channel_densities.append(kfast_channel_density)

    # kslow_channel_density = ChannelDensity(id="kslow_channels", cond_density="475.82 S_per_m2", erev="-80 mV", ion="k", ion_channel="kslow")
    # mem_prop.channel_densities.append(kslow_channel_density)

    # nat_channel_density = ChannelDensity(id="nat_channels", cond_density="236.616175 S_per_m2", erev="55 mV", ion="na", ion_channel="nat")
    # mem_prop.channel_densities.append(nat_channel_density)

    # nap_channel_density = ChannelDensity(id="nap_channels", cond_density="1.44 S_per_m2", erev="55 mV", ion="na", ion_channel="nap")
    # mem_prop.channel_densities.append(nap_channel_density)

    # km_channel_density = ChannelDensity(id="km_channels", cond_density="10.459916 S_per_m2", erev="-80 mV", ion="k", ion_channel="km")
    # mem_prop.channel_densities.append(km_channel_density)

    # # Other membrane properties
    # mem_prop.spike_threshes.append(SpikeThresh(value="-20mV"))
    # mem_prop.specific_capacitances.append(SpecificCapacitance(value="2.23041 uF_per_cm2"))
    # mem_prop.init_memb_potentials.append(InitMembPotential(value="-65mV"))

    # intra_prop = IntracellularProperties()
    # intra_prop.resistivities.append(Resistivity(value="0.082 kohm_cm"))

    # # Add to biological properties
    # bio_prop.intracellular_properties = intra_prop

    # # Morphology
    # morph = Morphology(id="pyr_cell_morph")
    # #  notes="Simple morphology for the HH cell")
    # seg = Segment(id="0", name="soma", notes="Soma segment")
    # # We want a diameter such that area is 1000 micro meter^2
    # # surface area of a sphere is 4pi r^2 = 4pi diam^2
    # # diam = math.sqrt(1682 / math.pi)
    # proximal = Point3DWithDiam(x="0", y="0", z="0", diameter=str(23.1453))
    # distal = Point3DWithDiam(x="0", y="23.1453", z="0", diameter=str(23.1453))
    # seg.proximal = proximal
    # seg.distal = distal
    # morph.segments.append(seg)
    # pyr_cell.morphology = morph

    pyr_cell_doc.cells.append(cell)
    pynml.write_neuroml2_file(nml2_doc=pyr_cell_doc, nml2_file_name=nml_cell_file, validate=True)
    return nml_cell_file 

create_pyr_cell()

pyNeuroML >>> INFO - Executing: (java -Xmx400M  -jar  "/Users/shayan/opt/anaconda3/envs/neuron/lib/python3.10/site-packages/pyneuroml/lib/jNeuroML-0.12.0-jar-with-dependencies.jar" -validate "pyr.cell.nml" ) in directory: .
pyNeuroML >>> INFO - Command completed. Output: 
  jNeuroML >>   jNeuroML v0.12.0
  jNeuroML >>  Validating: /Users/shayan/Desktop/OSB/BahlEtAl2012_ReducedL5PyrCell/NeuroML2/pyr.cell.nml
  jNeuroML >>  Valid against schema and all tests
  jNeuroML >>  No warnings
  jNeuroML >>  
  jNeuroML >>  Validated 1 files: All valid and no warnings
  jNeuroML >>  
  jNeuroML >>  


'pyr.cell.nml'

In [1]:
def create_pyr_network():
    """Create the network

    :returns: name of network nml file
    """
    net_doc = NeuroMLDocument(id="network",
                              notes="Pyramidal cell network")
    net_doc_fn = "pyr_example_net.nml"
    net_doc.includes.append(IncludeType(href=create_cell()))
    # Create a population: convenient to create many cells of the same type
    pop = Population(id="pop0", notes="A population for pyramidal cell", component="pyr_cell", size=1)
    # Input
    pulsegen = PulseGenerator(id="pg", notes="Simple pulse generator", delay="100ms", duration="500ms", amplitude="0.4nA")

    exp_input = ExplicitInput(target="pop0[0]", input="pg")

    net = Network(id="single_pyr_cell_network",
                  type="networkWithTemperature",
                  temperature = "37 degC",
                  note="A network with a single population")
    net_doc.pulse_generators.append(pulsegen)
    net.explicit_inputs.append(exp_input)
    net.populations.append(pop)
    net_doc.networks.append(net)

    pynml.write_neuroml2_file(nml2_doc=net_doc, nml2_file_name=net_doc_fn, validate=True)
    return net_doc_fn

In [2]:
if __name__ == "__main__":
    main()

NameError: name 'main' is not defined